In [22]:
pip install langchain langchain-community sentence-transformers chromadb


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [23]:
from pathlib import Path
import json

from langchain_community.chat_models import ChatOllama
from langchain_community.embeddings import SentenceTransformerEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_core.documents import Document


# Modelo de Ollama que quieres usar
LLM_MODEL_NAME = "deepseek-r1:8b"   # o "llama3:8b" si prefieres

# Rutas a tus JSON (ajusta si están en otra carpeta)
DATA_DIR = Path("../.json")
FINALS_PATH  = DATA_DIR / "finals.json"
HISTORY_PATH = DATA_DIR / "history.json"
PLAYERS_PATH = DATA_DIR / "players.json"
TEAMS_PATH   = DATA_DIR / "teams.json"
COACHES_PATH = DATA_DIR / "coaches.json"


In [24]:
def load_json_list(path: Path):
    with path.open("r", encoding="utf-8") as f:
        return json.load(f)

finals_data  = load_json_list(FINALS_PATH)
history_data = load_json_list(HISTORY_PATH)
players_data = load_json_list(PLAYERS_PATH)
teams_data   = load_json_list(TEAMS_PATH)
coaches_data = load_json_list(COACHES_PATH)

len(finals_data), len(history_data), len(players_data), len(teams_data), len(coaches_data)


(52, 21, 29, 21, 22)

In [25]:
def build_docs(data_list, source_name: str):
    docs = []

    for i, item in enumerate(data_list):
        categoria = item.get("categoria", source_name)
        tipo = item.get("tipo", "")
        doc_id = item.get("id", f"{source_name}_{i}")

        # Texto que vamos a indexar
        if tipo == "qa":
            q = item.get("question", "")
            a = item.get("answer", "")
            text_parts = []
            if q:
                text_parts.append("Pregunta: " + q)
            if a:
                text_parts.append("Respuesta: " + a)
            text = "\n".join(text_parts)
        else:
            titulo = item.get("titulo", "")
            texto = item.get("texto", "")
            if titulo and texto:
                text = f"{titulo}. {texto}"
            else:
                text = titulo or texto

        if not text:
            continue

        metadata = {
            "source": source_name,
            "categoria": categoria,
            "tipo": tipo,
            "id": doc_id,
            "season_year": item.get("season_year"),
        }

        docs.append(Document(page_content=text, metadata=metadata))

    return docs


docs_finals  = build_docs(finals_data,  "finals")
docs_history = build_docs(history_data, "history")
docs_players = build_docs(players_data, "players")
docs_teams   = build_docs(teams_data,   "teams")
docs_coaches = build_docs(coaches_data, "coaches")

all_docs = docs_finals + docs_history + docs_players + docs_teams + docs_coaches
len(all_docs)


145

In [26]:
all_docs[0]

Document(metadata={'source': 'finals', 'categoria': 'final', 'tipo': 'fact', 'id': 'ucl_1992_93_final_marseille_milan', 'season_year': 1993}, page_content='La primera final de la era Champions League, en la temporada 1992-93, enfrento a Marseille y Milan en Munich. Un gol de cabeza de Basile Boli dio el 1-0 a los franceses y supuso el primer titulo de la maxima competicion para un club frances.')

In [34]:
from sentence_transformers import SentenceTransformer

# (opción 1) directamente con langchain_community:
embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

# Vector store en memoria
vectorstore = Chroma.from_documents(
    documents=all_docs,
    embedding=embeddings,
)

# Retrievers por categoría usando el mismo vectorstore + metadata.source

retriever_all = vectorstore.as_retriever(search_kwargs={"k": 5})

retriever_history = vectorstore.as_retriever(
    search_kwargs={"k": 5, "filter": {"source": "history"}}
)

retriever_finals = vectorstore.as_retriever(
    search_kwargs={"k": 5, "filter": {"source": "finals"}}
)

retriever_players = vectorstore.as_retriever(
    search_kwargs={"k": 8, "filter": {"source": "players"}}
)

retriever_teams = vectorstore.as_retriever(
    search_kwargs={"k": 8, "filter": {"source": "teams"}}
)

retriever_coaches = vectorstore.as_retriever(
    search_kwargs={"k": 8, "filter": {"source": "coaches"}}
)



In [35]:
def choose_retriever(question: str):
    q = question.lower()

    # Heurísticos muy simples basados en palabras clave
    if any(word in q for word in ["equipo", "club", "equipos", "clubes"]):
        return retriever_teams

    if any(word in q for word in ["entrenador", "tecnico", "míster", "mister", "coach"]):
        return retriever_coaches

    if any(word in q for word in ["jugador", "goleador", "delantero", "portero", "defensa", "mediocentro"]):
        return retriever_players

    if any(word in q for word in ["final", "semifinal", "cuartos", "octavos", "eliminatoria"]):
        return retriever_finals

    if any(word in q for word in ["historia", "origen", "formato", "regla", "reglas", "cuando se", "cuándo se"]):
        return retriever_history

    # Si no sabemos, usamos el general
    return retriever_all


In [36]:
LLM_MODEL_NAME = "deepseek-r1:8b"  # o "llama3:8b" si quieres

llm = ChatOllama(
    model=LLM_MODEL_NAME,
    # temperature=0.2,  # opcional
)



In [37]:
def ask_champions(question: str):
    # 0) Elegir retriever según la pregunta
    retr = choose_retriever(question)

    # 1) Recuperar documentos relevantes
    docs = retr.invoke(question)

    # Reordenar: primero FACT, luego QA (y otros tipos al final)
    def sort_key(doc):
        tipo = doc.metadata.get("tipo", "")
        if tipo == "fact":
            return 0
        elif tipo == "qa":
            return 1
        else:
            return 2

    docs = sorted(docs, key=sort_key)

    print("📚 DOCUMENTOS RECUPERADOS (debug):\n")
    for d in docs:
        meta = d.metadata
        print(
            f"- source={meta.get('source')} | categoria={meta.get('categoria')} "
            f"| tipo={meta.get('tipo')} | id={meta.get('id')} | season_year={meta.get('season_year')}"
        )
        print("  Extracto:", d.page_content[:180].replace("\n", " "), "...\n")

    # 2) Construir el contexto
    context_parts = []
    for d in docs:
        meta = d.metadata
        header = f"[source={meta.get('source')} | categoria={meta.get('categoria')} | tipo={meta.get('tipo')} | id={meta.get('id')}]"
        context_parts.append(header + "\n" + d.page_content)

    context_text = "\n\n---\n\n".join(context_parts)

    prompt = f"""
Eres un asistente experto en la UEFA Champions League.
Dispones de un dataset estructurado con información estadística y descriptiva sobre equipos, jugadores, entrenadores, historia y finales.

Debes responder SOLO usando la información del CONTEXTO proporcionado.
La respuesta debe ser coherente **según estos datos**, no según conocimientos externos.
Si la respuesta no se encuentra claramente en el contexto, di explícitamente que no aparece en el dataset.

CONTEXTO (fragmentos del dataset):
{context_text}

PREGUNTA DEL USUARIO:
{question}

Responde en español, de forma clara y concisa.
""".strip()

    # 3) Llamar al modelo de Ollama
    response = llm.invoke(prompt)

    print("\n💬 RESPUESTA:\n")
    print(response.content)

    return response.content


In [38]:
ask_champions("¿Quién es el máximo goleador de la Champions según estos datos?")
ask_champions("¿Qué equipo ha ganado más veces la Champions en este dataset?")

📚 DOCUMENTOS RECUPERADOS (debug):

- source=players | categoria=player | tipo=qa | id=players_qa_005_midfield_vs_forwards | season_year=None
  Extracto: Pregunta: Como complementan los centrocampistas creativos a los grandes goleadores en la Champions? Respuesta: Jugadores como Xavi, Iniesta, De Bruyne o Verratti no siempre lideran ...

- source=players | categoria=player | tipo=qa | id=players_qa_005_midfield_vs_forwards | season_year=None
  Extracto: Pregunta: Como complementan los centrocampistas creativos a los grandes goleadores en la Champions? Respuesta: Jugadores como Xavi, Iniesta, De Bruyne o Verratti no siempre lideran ...

- source=players | categoria=player | tipo=qa | id=players_qa_005_midfield_vs_forwards | season_year=None
  Extracto: Pregunta: Como complementan los centrocampistas creativos a los grandes goleadores en la Champions? Respuesta: Jugadores como Xavi, Iniesta, De Bruyne o Verratti no siempre lideran ...

- source=players | categoria=player | tipo=qa | id=pl

'\nEl Bayern Munich ha ganado 3 veces la UEFA Champions League, en los años 2001, 2013 y 2020.'

In [39]:
ask_champions("¿Quién fue el entrenador del Real Madrid en la final de 2014 según estos datos?")

📚 DOCUMENTOS RECUPERADOS (debug):

- source=coaches | categoria=coach | tipo=fact | id=coach_diego_simeone_profile | season_year=None
  Extracto: Perfil historico de Diego Simeone en la Champions League. Diego Simeone es uno de los entrenadores mas competitivos de la Champions moderna. Ha dirigido 82 partidos con el Atletico ...

- source=coaches | categoria=coach | tipo=fact | id=coach_diego_simeone_profile | season_year=None
  Extracto: Perfil historico de Diego Simeone en la Champions League. Diego Simeone es uno de los entrenadores mas competitivos de la Champions moderna. Ha dirigido 82 partidos con el Atletico ...

- source=coaches | categoria=coach | tipo=fact | id=coach_diego_simeone_profile | season_year=None
  Extracto: Perfil historico de Diego Simeone en la Champions League. Diego Simeone es uno de los entrenadores mas competitivos de la Champions moderna. Ha dirigido 82 partidos con el Atletico ...

- source=coaches | categoria=coach | tipo=qa | id=coach_qa_001_mas_champio

'\nNo aparece en el dataset.'

In [40]:
ask_champions("¿Que jugador tiene más apariciones en la Champions League según estos datos?")

📚 DOCUMENTOS RECUPERADOS (debug):

- source=players | categoria=player | tipo=fact | id=players_overview_top_assisters | season_year=None
  Extracto: Jugadores con mas asistencias en la Champions segun el dataset. Ademas de los goleadores, la base de datos recoge a los grandes asistentes de la era moderna. Segun los datos de acc ...

- source=players | categoria=player | tipo=fact | id=players_overview_top_assisters | season_year=None
  Extracto: Jugadores con mas asistencias en la Champions segun el dataset. Ademas de los goleadores, la base de datos recoge a los grandes asistentes de la era moderna. Segun los datos de acc ...

- source=players | categoria=player | tipo=fact | id=players_overview_top_assisters | season_year=None
  Extracto: Jugadores con mas asistencias en la Champions segun el dataset. Ademas de los goleadores, la base de datos recoge a los grandes asistentes de la era moderna. Segun los datos de acc ...

- source=players | categoria=player | tipo=fact | id=players_o

'\nNo hay información disponible en el dataset sobre el número de apariciones de los jugadores en la UEFA Champions League.'